In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import os
import numpy as np
import scipy
import glob
import sys
from scipy import io
import cv2
import math
import random
import keras
from sklearn import preprocessing

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# GLOBALS

In [2]:
POET_DIR = "D:/data/POETdataset/"
PATCHES_FILE = os.path.join(POET_DIR, "x.npy")
LABELS_FILE = os.path.join(POET_DIR, 'y.npy')
pascal_images = os.path.join(POET_DIR, 'PascalImages')
PATCH_SIZE = 32
HALF_PATCH = PATCH_SIZE//2
DATASET_SIZE = 6270
VALIDATION_PERC = 0.1
MAX_GAZE_POINTS = 3 # TEMPORARY
MIN_GAZE_POINTS = 3
VGG_NR_FEATURES = 512
user_index = 0 # TEMPORARY
BATCH_SIZE = 50

In [3]:
os.chdir(POET_DIR)

# PATCH EXTRACTION 

In [4]:
classes = [a.split("_")[1].split(".mat")[0] for a in glob.glob("etData/*")]
idx2class = {i:c for i, c in enumerate(classes)}
class2idx = {c:i for i, c in enumerate(classes)}

In [5]:
def fixation_within_image(fx, fy, dims):
#   print(dims)
  if fx < 0:
#     print(fx, fy, 'not within')
    return False
  if fx > dims[0]:
#     print(fx, fy, 'not within')
    return False
  if fy < 0:
#     print(fx, fy, 'not within')
    return False
  if fy > dims[1]:
#     print(fx, fy, 'not within')
    return False

  return True

In [6]:
def get_avg_fixations(fixR, fixL, dims):
  # ger average of fixations between right and left eye
  fix = []
  for i in range(max(len(fixR),3)):
    fR = fixR[i]
    fL = fixL[i]
    # no fixations outside
    fx = np.mean([fR[0],fL[0]])
    fy = np.mean([fR[1],fL[1]])
    if fixation_within_image(fx, fy, dims):
      fix.append([fx,fy])

  fix = np.array(fix)
  return fix[:3]

In [7]:
def get_fixations(filename, classname, dims):
  filename = 'sofa_2010_004095'
  classname = 'sofa'
  if classname in filename:
    filename = filename.split("%s_" %classname)[1]
  c_instances = io.loadmat(os.path.join(POET_DIR,'etData','etData_%s.mat' %classname), squeeze_me=True)['etData']
  user_index = 1
  for i in c_instances:
    if filename == i['filename']:
      fixR = i['fixations'][user_index]['imgCoord']['fixR'].tolist()['pos'].tolist()
      fixL = i['fixations'][user_index]['imgCoord']['fixL'].tolist()['pos'].tolist()
      fix = get_avg_fixations(fixR, fixL, dims)
      return fix


In [8]:
def gen_patches(filename, classname):
  if classname in filename:
    filename = filename.split("%s_" %classname)[1]

  img_src = cv2.imread(os.path.join(POET_DIR, 'PascalImages', '%s_%s.jpg' %(classname, filename)), )
#   print('img shape = ', img_src.shape)
#   plt.imshow(img_src)

  img = np.zeros((img_src.shape[0]+PATCH_SIZE, img_src.shape[1]+PATCH_SIZE, 3), dtype=int)
  img[HALF_PATCH:HALF_PATCH+img_src.shape[0],HALF_PATCH:img_src.shape[1]+HALF_PATCH] = img_src
#   plt.imshow(img)

  fix = get_fixations(filename, classname, (img_src.shape[0], img_src.shape[1]))
#   print(fix)
  if len(fix) < MIN_GAZE_POINTS:
    return None

  patches = np.zeros((len(fix), PATCH_SIZE, PATCH_SIZE, 3), dtype=int)

  for i, f in enumerate(fix):
    fx = math.floor(f[0]) + HALF_PATCH # to account for paddings
    fy = math.floor(f[1]) + HALF_PATCH
    p = img[fx-HALF_PATCH:fx+HALF_PATCH,fy-HALF_PATCH:fy+HALF_PATCH]
    assert p.shape==(PATCH_SIZE, PATCH_SIZE, 3), 'file "%s" of class %s has a patch of shape %s' %(filename, classname, p.shape)
    patches[i] = p
  return patches

# DATA GENERATOR

In [9]:
# CNN for extracting features of a patch
vgg16 = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(32,32,3), pooling=None, classes=10)

In [10]:
partition = []
labels = {}

In [11]:
for classname in classes:
  c_instances = io.loadmat(os.path.join(POET_DIR,'etData','etData_%s.mat' %classname), squeeze_me=True)['etData']
  for c_instance in c_instances:
    # id
    filename = "%s_%s" %(classname, c_instance['filename'])
    label = class2idx[classname]
    partition.append(filename)
    if filename in labels.keys():
      print(filename)
    labels[filename] = label

In [12]:
del c_instances

In [13]:
assert len(partition)==DATASET_SIZE
assert len(list(labels.keys()))==DATASET_SIZE

In [14]:
random.shuffle(partition)
valid_size = VALIDATION_PERC*len(partition)
validation = partition[:int(valid_size)]
train = partition[int(valid_size):]
assert len(validation)+ len(train)==DATASET_SIZE

In [15]:
partition = {
  'train':train,
  'validation':validation
}

In [16]:
np.save(LABELS_FILE, labels)

In [17]:
np.save('idx2class.npy', idx2class)

In [18]:
np.save('class2idx.npy', class2idx)

# STORE ALL FEATURES OF ALL PATCHES

In [19]:
# key - 'class_filename', value - array of patches (MAX_GAZE_POINTS, VGG_NR_FEATURES)
storage = {}

In [ ]:
for split in ['train', 'validation']:
  ids = partition[split]
  dropped = 0
  for i, filename in enumerate(ids):
    if filename not in storage.keys():
      classname = idx2class[labels[filename]]
      patches = gen_patches(filename, classname)
      if patches is not None:
        p_features = vgg16.predict(patches)
        storage[filename] = p_features[:MAX_GAZE_POINTS,0,0:]
      else:
        dropped += 1
print('dropped %s' %dropped)

In [ ]:
np.save(PATCHES_FILE, storage)